<a href="https://colab.research.google.com/github/fdouradof/SML/blob/main/TeacherBOT_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediawikiapi
!pip install wikipedia

In [171]:
import pandas as pd
import numpy as np
import mediawikiapi
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn import metrics
import wikipedia
import requests
from bs4 import BeautifulSoup
SIZE = 10
def dificuldade(x):
    if x < -2:
        return 2

    if x > -2 and x < -1:
        return 1

    else:
        return 0
def evaluate(model,x_test,y_test):
  y_pred = model.predict(x_test)
  conf_mat = metrics.confusion_matrix(y_test,y_pred)
  print(conf_mat)

  accuracy = metrics.accuracy_score(y_test,y_pred)
  sensitivity = metrics.recall_score(y_test,y_pred,average = None)
  precision = metrics.precision_score(y_test,y_pred, average = None)
  print('Accuracy    = {}'.format(np.round(accuracy,3)))
  print('Sensitvity  = {}'.format(np.round(sensitivity,3)))
  print('Precision   = {}'.format(np.round(precision,3)))
  print("       ")
def ti(docs,max_features,min_df,max_df):
    docs = [str(x).lower() for x in docs]
    vectorizer = TfidfVectorizer(max_features = max_features, min_df = min_df, max_df = max_df, stop_words=stopwords.words('english'))
    X = vectorizer.fit_transform(docs).toarray()
    key_words = vectorizer.get_feature_names_out()
    return X, key_words, vectorizer

def read_data(filename):
    df = pd.DataFrame(pd.read_excel(filename))
    #mask = df.loc[df['Category'] == "Info"]
    model_data = df[["Excerpt","BT Easiness"]]
    model_data["label"] = model_data["BT Easiness"].apply(dificuldade)
    return model_data
def predict(new_docs):
    new_line_X  = my_vectorizer.transform(new_docs)
    return GNB.predict(new_line_X.toarray())


def query(query):
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'list': 'search',
        'utf8': 1,
        'srsearch': query
    }

    data = requests.get(url, params=params).json()

    for i in data['query']['search']:
        print(i['title'], ' - Word count: ', i['wordcount'])
    return data
def links(subject):
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'parse',
        'prop': 'text',
        'format': 'json',
        'page': subject,
        'section': 0,
        'redirects': ''
    }
    data = requests.get(url, params=params).json()

    soup = BeautifulSoup(data['parse']['text']['*'], 'html.parser')

    infobox = soup.find('table', {'class': 'infobox', 'class': 'sidebar'})

    a_tags = infobox.find_all('a', href=True)

    links = []
    for tag in a_tags:
        if not tag.text == '' and 'wiki' in tag['href']:
            links.append({'anchor': tag.text, 'href': tag['href']})

    return pd.DataFrame(links)
def text(subject):
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'parse',
        'page': subject,
        'format': 'json',
        'prop': 'text',
        'redirects': ''
    }

    response = requests.get(url, params=params)
    data = response.json()

    raw_html = data['parse']['text']['*']
    soup = BeautifulSoup(raw_html, 'html.parser')
    soup.find_all('p')
    text = ''

    for p in soup.find_all('p'):
        text += p.text

    # print('Text length: ', len(text))
    return text


def wordize(soup,match):

    page_title = "TITLE"
    # find("span", class_="mw-page-title-main")
    page_texts = soup.find_all("p")

    text_f = []
    words = []
    data = []
    for text in page_texts:
        text_t = nltk.word_tokenize(text.text)
        # if match in text_t:
        words.append((text_t))
        text_f.append(text.text)
        sentences = nltk.sent_tokenize(text.text)
        for i in range(len(sentences)):
            sentences[i] = sentences[i].lower()
            sentences[i] = re.sub(r'\W', ' ',
                                  sentences[i])  # Substitute the non-alphanumeric characters with space.
            sentences[i] = re.sub(r'\s+', ' ', sentences[i])  # Remove the excess of white spaces.
            sentences[i] = re.sub(r'\s$', '', sentences[i])  # Remove the space at the end of a sentence.
        for i in range(len(sentences)):
            words = nltk.word_tokenize(sentences[i])  # Tokenize into words.
            words = [x for x in words if x not in stopwords.words('english')]  # Remove the stop words.
            sentences[i] = ' '.join(words)


        data.append(sentences)


    if len(data)<4:
        return False

    else:
        print("Analizing Page title:     ",page_title)
        print("paragraphs matches (LEN): ",len(data))
        print("-------------------------------------")
        return(data)
def get_links(soup,word):
    lk = np.zeros(SIZE, dtype= "U100")
    lkt = np.zeros(SIZE, dtype= "U100")
    count = 0

    for link in soup.find_all("a"):
        url = link.get("href", "")

        if word in url.lower():
            try:
                lk[count] = url
                lkt[count] = link.text.strip()
                count += 1
            except:
                pass

    df = {"Title" : lkt , "Link" : lk}
    df = pd.DataFrame(df)
    return df
def sciencekids():
    "retorna lista de textos para serem preditos"
    page = requests.get("https://www.sciencekids.co.nz/lessonplans/chemistry.html")
    soup = BeautifulSoup(page.content, "html.parser")
    s_links = get_links(soup, "chemistry")
    catalogo = s_links[::2]
    texts = []
    for link in catalogo["Link"]:
        url = "https://www.sciencekids.co.nz/lessonplans/" + link
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        text = "".join(str(wordize(soup, "egg")))
        texts.append(text)
    return texts
def role():
    txt = []
    titles = []
    for i in range(len(q)):
        title = q[i]
        txt.append(text(title))
        titles.append(title)

    print(list(zip(titles, predict(q))))

# texto = sciencekids()
# print(predict(texto))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [172]:
#READ DATA
modeldata = read_data("Clear Corpus 6.0.1 (1).xlsx")
docs = modeldata["Excerpt"]

#VECTORIZER
X, kw, my_vectorizer = ti(docs,max_features=2000, min_df=0.01, max_df=0.90)
x_values = pd.DataFrame(X, columns = kw).values

y_values = modeldata["label"].values
#ML MODELS
M = LogisticRegression(solver='liblinear',max_iter=53)
x_train,x_test, y_train, y_test = train_test_split(x_values,y_values, test_size = 0.33, random_state=42)
M.fit(x_train,y_train)
GNB = GaussianNB()
GNB.fit(x_train,y_train)
KNN = KNeighborsClassifier(n_neighbors=51)
KNN.fit(x_train,y_train)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KNeighborsClassifier(n_neighbors=51)

In [182]:
import ipywidgets
from ipywidgets import interact, widgets
from IPython.display import display

# take user input
page_texts = []
def submit(a):
    print('Pesquisa aceite com', text_wid.value)
    search = wikipedia.search(text_wid.value)
    print("PESQUISA: ",search)
    for title in search:
      page_text = text(title)
      page_texts.append(page_text)
      # print("TEXTO: ", page_text[:100] )
    return page_texts

text_wid = ipywidgets.Text(value='ML',placeholder='escolha algo interessante',description='Query:',disabled=False)
instructions = ipywidgets.widgets.HTML("Wikipedia Search")
display(instructions)
display(text_wid)
text_wid.on_submit(submit)

HTML(value='Wikipedia Search')

Text(value='ML', description='Query:', placeholder='escolha algo interessante')

Pesquisa aceite com Machi
PESQUISA:  ['Machi', 'Erased (manga)', 'Carmen Machi', 'Machi Koro', 'Machi (name)', 'Super Machi', 'Nankin-machi', 'Machi Goth', 'Machi (film)', 'Machi (shaman)']
Pesquisa aceite com Machine
PESQUISA:  ['Machine', 'Wayback Machine', 'Machine learning', 'Machine to machine', "Gary Dell'Abate", 'Machine gun', 'Simple machine', 'Sewing machine', 'Machine II Machine', 'Enigma machine']
Pesquisa aceite com Cooking
PESQUISA:  ['Cooking', 'Induction cooking', 'Cooking banana', 'Cooking oil', 'Cooking? Cooking!', 'Molecular gastronomy', 'Blanching (cooking)', 'Pressure cooking', 'Batter (cooking)', 'Smoking (cooking)']


In [186]:
def on_button_clicked(a):
  z = zip(wikipedia.search(text_wid.value),predict(page_texts))
  print(list(z))

button = ipywidgets.widgets.Button(description="Classificar")
button.on_click(on_button_clicked)
display(button)

Button(description='Classificar', style=ButtonStyle())

[('Machine', 2), ('Wayback Machine', 2), ('Machine learning', 0), ('Machine to machine', 2), ("Gary Dell'Abate", 2), ('Machine gun', 1), ('Simple machine', 1), ('Sewing machine', 2), ('Machine II Machine', 1), ('Enigma machine', 2)]
[('Cooking', 2), ('Induction cooking', 2), ('Cooking banana', 0), ('Cooking oil', 2), ('Cooking? Cooking!', 2), ('Molecular gastronomy', 1), ('Blanching (cooking)', 1), ('Pressure cooking', 2), ('Batter (cooking)', 1), ('Smoking (cooking)', 2)]
